In [ ]:
!pip install autogluon

In [ ]:
!wget https://raw.githubusercontent.com/Western-OC2-Lab/Intrusion-Detection-System-Using-Machine-Learning/refs/heads/main/data/CICIDS2017_sample.csv

In [ ]:
!unzip /content/MachineLearningCSV.zip

In [ ]:
# https://auto.gluon.ai/stable/index.html
from autogluon.tabular import TabularDataset, TabularPredictor

data = '/content/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
# data = 'https://raw.githubusercontent.com/Western-OC2-Lab/Intrusion-Detection-System-Using-Machine-Learning/refs/heads/main/data/CICIDS2017_sample.csv'

data = TabularDataset(data)
train_data = data.sample(frac=0.9, random_state=42)
test_data = data.drop(train_data.index)

In [ ]:
predictor = TabularPredictor(label='Label').fit(train_data=train_data, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels/ag-20250117_214103"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.15 GB / 12.67 GB (88.0%)
Disk Space Avail:   75.64 GB / 107.72 GB (70.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions 

In [ ]:
print(predictor.leaderboard(test_data))
print(len(test_data))
predictions = predictor.predict(test_data)
print(len(test_data[predictions == test_data['Label']]))

print(predictor.evaluate(test_data))

/usr/local/lib/python3.11/dist-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:188: RuntimeWarning: invalid value encountered in subtract
  (X[self.cont_columns].values - cont_mean) / cont_std,


                 model  score_test  score_val eval_metric  pred_time_test  \
0             LightGBM    0.998235     0.9984    accuracy        0.113173   
1  WeightedEnsemble_L2    0.998235     0.9984    accuracy        0.116729   
2           LightGBMXT    0.996294     0.9980    accuracy        0.411423   
3        LightGBMLarge    0.995588     0.9980    accuracy        0.105234   
4             CatBoost    0.992234     0.9936    accuracy        0.048311   
5      NeuralNetFastAI    0.412813     0.4000    accuracy        0.108102   

   pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0       0.054151  14.067505                 0.113173                0.054151   
1       0.055601  14.159347                 0.003556                0.001450   
2       0.174577  18.099866                 0.411423                0.174577   
3       0.024274  14.854946                 0.105234                0.024274   
4       0.015398  55.821312                 0.048311        

In [ ]:
# repeatedly taking new test and train sets and
# training a different predictor for each and averaging accuracy

import random
import pandas as pd

def train_test_on_new():
  train_data = data.sample(frac=0.9, random_state=random.randint(0, 1000))
  test_data = data.drop(train_data.index)
  predictor = TabularPredictor(label='Label', learner_kwargs={"cache_data": False}).fit(train_data=train_data, fit_strategy='parallel')
  return predictor.evaluate(test_data)['accuracy']

results = []
for i in range(10):
  result = train_test_on_new()
  print(result)
  results.append(result)

average = sum(results) / len(results)
print(f"average: {average}")

No path specified. Models will be saved in: "AutogluonModels/ag-20250118_022055"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.61 GB / 12.67 GB (83.7%)
Disk Space Avail:   75.24 GB / 107.72 GB (69.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions 

0.9977056124249912


	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 8 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 10): ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Featu

0.9955877162019061


	Available Memory:                    10990.86 MB
	Train Data (Original)  Memory Usage: 29.96 MB (0.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 8 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 10): ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature

...

0.9955877162019061


Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10983.47 MB
	Train Data (Original)  Memory Usage: 29.96 MB (0.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 8 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 10): ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
		These features

0.9955877162019061


Train Data Class Count: 7
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10976.88 MB
	Train Data (Original)  Memory Usage: 29.96 MB (0.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 8 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 10): ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bu

0.9955877162019061
average: 0.9957995058242146


In [ ]:
print(results)

[0.9977056124249912, 0.9955877162019061, 0.9955877162019061, 0.9955877162019061, 0.9955877162019061, 0.9955877162019061, 0.9955877162019061, 0.9955877162019061, 0.9955877162019061, 0.9955877162019061]
